In [34]:
# CONSIDER WHETHER SAMPLING REQUIRED BECAUSE OHC WILL INCREASE DATA SIZE SIGNIFICANTLY
import os
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin # for custom encoder for user tags
import gc # force garbage collection to free memory
from joblib import dump, load

In [35]:
SAVE_PROCESSED_DATA = True

In [3]:
# scriptDir = os.getcwd() # Assumes that script is executed from its actual location
# relPath = r"../data/" # And data is located in a 'data' folder in the same parent directory as 'code' folder
# trainFilePath = os.path.join(scriptDir, relPath,"train.csv")
# valFilePath = os.path.join(scriptDir, relPath,"validation.csv")
# sampleFilePath = os.path.join(scriptDir, relPath,"sample.csv")
# testFilePath = os.path.join(scriptDir, relPath,"test.csv")
# relPathOutput = r"../output/"
# outputFolderPath = os.path.join(scriptDir, relPathOutput)

In [36]:
# Load data
df = pd.read_csv('/Users/mac/Desktop/multi-agent AI group/train.csv')

In [37]:
print(df.shape)
df.head(5)

(2430981, 25)


,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2.0,...,200,2,0,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,NaN
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1.0,...,250,FourthView,Na,0,10722,294,23,NaN,2821,NaN
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2.0,...,250,2,0,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110"
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1.0,...,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111"
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2.0,...,90,OtherView,Na,133,7330,277,133,NaN,2259,NaN


In [6]:
# Split into features and labels
# List all columns and summarise what will be done with them
# In general, numerical will be scaled and categorical will be one-hot-encoded
# Some may be dropped
# Some additional features need to be created
# Some will be dropped

In [7]:
# click - LABEL
# weekday - categorical, encode
# hour - categorical, encode
# bidid - ignore
# userid - ignore
# useragent - ignore (but see OS and browser)
# IP - ignore
# region - categorical, encode
# city - categorical, encode
# adexchange - categorical, encode
# domain - ignore
# url - ignore
# urlid - ignore
# slotid - ignore
# slotwidth - numeric, scaled
# slotheight - numeric, scaled
# slotvisibility - categorical, encode
# slotformat - categorical, encode
# slotprice - numeric, scaled
# creative - ?
# bidprice - ignore (train/val only)
# payprice - ignore (train/val only)
# keypage - ?
# advertiser - categorical, encode
# usertag - categorical, parse+separate and encode
# OS - derived, categorical, encode
# browser - derived, categorical, encode
# slotsize - derived, numerical, scaled

In [8]:
# reduce dataset for testing
#df = df.head(10)

In [38]:
# Split 'useragent' field into 'OS' and 'browser'
dfn = df['useragent'].str.split('_', expand = True)
df['OS'] = dfn[0]
df['browser'] = dfn[1]
# Create slotsize field
# df['slotsize'] = df['slotheight'] * df['slotwidth']
#combine slotheight and slot width
df['slot_width_height'] = df['slotwidth'].apply(lambda x: str(x)) + "_" + df['slotheight'].apply(lambda x: str(x))
df.drop(['slotwidth', 'slotheight'], axis=1)

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,OS,browser,slot_width_height
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2.0,...,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,NaN,windows,ie,200_200
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1.0,...,0,10722,294,23,NaN,2821,NaN,windows,chrome,300_250
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2.0,...,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110",windows,ie,250_250
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1.0,...,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111",windows,ie,160_600
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2.0,...,133,7330,277,133,NaN,2259,NaN,windows,ie,728_90
5,0,4,17,1be2cf4a47a2a6aee4b0fa64d1b786d3897be4f0,8810fd690f88ddfb797867e3e08e7816f274bd46,windows_ie,119.45.182.*,276,277,1.0,...,0,77819d3e0b3467fe5c7b16d68ad923a1,300,123,bebefa5efe83beee17a3d245e7c5085b,1458,"10006,10077,10063,10057,10110",windows,ie,336_280
6,0,3,12,6b4f23dd215a5f11136c2f29214236cfee1f0122,fbd1bcaf2b33108d2c7f88ac41f0b0194b508a9b,windows_ie,112.82.93.*,80,89,3.0,...,20,86c2543527c86a893d4d4f68810a0416,300,20,43f4a37f42a7c5e6219e2601b26c6976,3386,"10083,10024,10063,13496,10093,10006,10057,10110",windows,ie,1000_90
7,0,3,16,f7c1d8ea177211249456c79e194617ce107bc077,9b539ba886146562a3991fd9f94bb9f9bad1f647,windows_ie,118.75.92.*,15,19,1.0,...,0,cb7c76e7784031272e37af8e7e9b062c,300,134,bebefa5efe83beee17a3d245e7c5085b,1458,"10063,10024,13800,13866,10006,10111",windows,ie,160_600
8,0,5,14,9e97694096fe8692851048a893231a7f7ff87922,ddc9c85b6a5726aafaa4c42239e90fb7a44a88d7,windows_ie,61.185.128.*,333,342,3.0,...,70,832b91d59d0cb5731431653204a76c0e,300,70,bebefa5efe83beee17a3d245e7c5085b,1458,"10063,10006,10083,10110",windows,ie,1000_90
9,0,5,0,d1b7c7fdb88ce3aa78a540bb29848dd77329a574,63b4f9a3638f81af361ab3c4190fd3e1b5e70fea,windows_chrome,117.9.211.*,2,2,3.0,...,70,a10c31a8ff5f42930b4c34035e523886,241,77,d29e59bf0f7f8243858b8183f14d4412,3358,"13403,10031,10063,10083,10006,10077,10075,10110",windows,chrome,1000_90


In [10]:
# convert numeric (non categorical) fields from int to float (to avoid warnings later)
# TODO

In [11]:
# d=df.keypage.unique()
# e=df.creative.unique()
# u=df.url.unique()
# dd=df.domain.unique()
# s=df.userid.unique()

In [12]:
# print(len(d))
# print(len(e))
# print(len(u))
# print(len(dd))
# print(len(s))


In [39]:
# split into features and labels##split later
X_train = df[['weekday', 'slot_width_height','hour', 'region', 'city', 'slotvisibility', 'slotformat', 'advertiser', 'OS', 'browser',
               'creative',
                  'slotprice', 
                 'usertag']]
y_train = df['click']
# # print(X_train.head(5))
# # print(y_train.head(5))

In [40]:
class UserTagEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.fitted = False

    def fit(self, X, y=None):
        #X = np.array(X).reshape(X.shape[0],)
        X = np.array(X)
        
        usertagsUnique = set()

        for idx in range(X.shape[0]):
            if type(X[idx,0]) != str:
                continue
            else:
                tags = str(X[idx,0]).split(',')

            for tag in tags:
                usertagsUnique.add(tag)

        usertagsUnique = list(usertagsUnique)
        columns = ['u0_'+x for x in usertagsUnique]
        
        self.tags = usertagsUnique
        self.columns = columns
        self.fitted = True

        # Return the transformer
        return self

    def transform(self, X):
        #X = np.array(X).reshape(X.shape[0],)
        X = np.array(X)
        
        # Check is fit had been called
        if not(self.fitted):
            raise ValueError("Estimator not fitted")

        # Input validation
        #X = check_array(X, accept_sparse=True)
        
        # already have the unique list
        usertagsUnique = self.tags
        # now get all the tags from the data provided and check against it
        X_new = np.zeros((X.shape[0],len(usertagsUnique)))
        for n in range(X.shape[0]):
            #print(n)
            if type(X[n,0]) != str:
                usertagsMulti = []
            else:
                usertagsMulti = str(X[n,0]).split(',')
        
            for m in range(len(usertagsUnique)):
                if usertagsUnique[m] in usertagsMulti:
                    X_new[n,m] = 1

        return X_new

In [41]:
# Pre-processing
# NOTE: adexchange removed from catgColumns since contains NaNs. TODO properly
# catg
oneHotEnc = OneHotEncoder(sparse=False, dtype=int,handle_unknown='ignore')
catgColumns = ['weekday', 'hour','slot_width_height', 'region', 'city', 'slotvisibility', 'slotformat', 'advertiser', 'OS', 'browser','creative']
catgTransform_info = ('catg', oneHotEnc, catgColumns)
# # num
# scale = StandardScaler()
# numColumns = ['slotwidth', 'slotheight', 'slotprice', 'slotsize']
# numTransform_info = ('num', scale, numColumns)
# usertags
tagEncoder = UserTagEncoder()
tagColumn = ['usertag']
tagTransform_info = ('tag', tagEncoder, tagColumn)
# column transformer for all
allPreProc = ColumnTransformer([ catgTransform_info, tagTransform_info])
# X_train_pp = allPreProc.fit_transform(df)

In [42]:
X_train_pp = allPreProc.fit_transform(df)

In [45]:
X_train_pp=pd.DataFrame(X_train_pp)
X_train_pp['slotprice']=df['slotprice']

In [56]:
# save estimator to disk to save time later if want to apply to other data
#dump(allPreProc, r"../output/allPreProc.joblib")
# allPreProc = load(r"../output/allPreProc.joblib")

['../output/allPreProc.joblib']

In [138]:
# example processed record
#X_train_pp[0,:]

In [47]:
# check dimensions
X_train_pp.shape

(2430981, 704)

In [52]:
X_train_pp

,0,1,2,3,4,5,6,7,8,9,...,694,695,696,697,698,699,700,701,702,slotprice
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,133
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70
9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70


In [53]:
#not enough space
# X_train_pp.to_csv('X_train_pp.csv')
# y_train.to_csv('Y_train.csv')

In [54]:
# column layout
allColumns = list()
# first columns are the scaled numeric ones, so just numColumns
# for idx in range(len(numColumns)):
#     allColumns.append(numColumns[idx])
for idx in range(len(catgColumns)):
    for subIdx in range(len(allPreProc.named_transformers_['catg'].categories_[idx])):
        allColumns.append('_'.join([str(catgColumns[idx]),str(allPreProc.named_transformers_['catg'].categories_[idx][subIdx])]))
for idx in range(len(allPreProc.named_transformers_['tag'].tags)):
    allColumns.append('_'.join(['usertag',allPreProc.named_transformers_['tag'].tags[idx]]))
#print(allColumns)
allColumns.append('slotprice')

In [55]:
#variable name
allColumns

['weekday_0',
 'weekday_1',
 'weekday_2',
 'weekday_3',
 'weekday_4',
 'weekday_5',
 'weekday_6',
 'hour_0',
 'hour_1',
 'hour_2',
 'hour_3',
 'hour_4',
 'hour_5',
 'hour_6',
 'hour_7',
 'hour_8',
 'hour_9',
 'hour_10',
 'hour_11',
 'hour_12',
 'hour_13',
 'hour_14',
 'hour_15',
 'hour_16',
 'hour_17',
 'hour_18',
 'hour_19',
 'hour_20',
 'hour_21',
 'hour_22',
 'hour_23',
 'slot_width_height_1000_90',
 'slot_width_height_120_240',
 'slot_width_height_120_600',
 'slot_width_height_125_125',
 'slot_width_height_160_600',
 'slot_width_height_180_150',
 'slot_width_height_180_600',
 'slot_width_height_200_200',
 'slot_width_height_234_60',
 'slot_width_height_250_230',
 'slot_width_height_250_250',
 'slot_width_height_300_100',
 'slot_width_height_300_250',
 'slot_width_height_300_300',
 'slot_width_height_300_600',
 'slot_width_height_320_50',
 'slot_width_height_336_280',
 'slot_width_height_360_300',
 'slot_width_height_468_60',
 'slot_width_height_620_60',
 'slot_width_height_640_90',

In [57]:
len(allColumns)

704

In [ ]:
# # *** clear training data from memory before continuing ***
# del df
# # del X_train
# del X_train_pp
# del y_train
# gc.collect()

# Load and run on validation data

In [58]:
dfVal = pd.read_csv('/Users/mac/Desktop/multi-agent AI group/validation.csv')

In [59]:
# Split 'useragent' field into 'OS' and 'browser'
dfn = dfVal['useragent'].str.split('_', expand = True)
dfVal['OS'] = dfn[0]
dfVal['browser'] = dfn[1]
# Create slotsize field
# dfVal['slotsize'] = dfVal['slotheight'] * dfVal['slotwidth']
dfVal['slot_width_height'] = dfVal['slotwidth'].apply(lambda x: str(x)) + "_" + dfVal['slotheight'].apply(lambda x: str(x))
dfVal.drop(['slotwidth', 'slotheight'], axis=1)

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,OS,browser,slot_width_height
0,0,4,20,bbcb813b6166538503d8b33a5602d7d72f6019dc,663169f66491c98c69f1f94a5c48fa34aa9fe06f,windows_ie,211.144.203.*,79,79,1.0,...,0,cb7c76e7784031272e37af8e7e9b062c,300,23,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10111",windows,ie,160_600
1,0,1,21,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,a23f0cfab6592c137f796e68fa752fceba08bb1b,windows_chrome,58.247.250.*,79,79,1.0,...,0,c938195f9e404b4f38c7e71bf50263e5,238,75,a3becb967c015e9acb5e72f22b100ef8,3476,"10075,10057,10024,10052,10083,10063,10006,10110",windows,chrome,950_90
2,0,4,8,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5105a013ea54a9706146033e6a138d5234a3803e,windows_ie,117.12.111.*,2,2,2.0,...,5,911b2d84826786018761e8c0b0a3a60c,238,65,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10024,10059,10063,13776,10083,10006,10111",windows,ie,300_250
3,0,5,15,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,577e9a768ce17e73c61ba7022db927f9ee761830,windows_chrome,61.187.224.*,201,205,2.0,...,5,dc0998c10f8f0b623b5d949e8272e4c7,238,6,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10057,10006,10063,10110",windows,chrome,336_280
4,0,1,18,0899bf144249458ea9c89188473694bf44c7ca15,e4bf79c562745d671b19f2edf7fda89c2e25987f,windows_chrome,117.41.145.*,134,135,2.0,...,5,23485fcd23122d755d38f8c89d46ca56,249,5,7e0e909ea89f707bc20f9585c9072081,3476,"10006,10063,13776,10110",windows,chrome,200_200
5,0,6,1,f4c6a9a3b1db6da035c7e2a36d4f5e389095beca,5d3f7a140423324201f980609348422b7b9d2ba8,windows_ie,101.23.72.*,3,3,3.0,...,20,3d8f1161832704a1a34e1ccdda11a81e,241,22,b2e35064f3549d447edbbdfb1f707c8c,3427,"10063,10111",windows,ie,960_90
6,0,0,18,786a2940f225482dc04d455dc7a7fea436b02e03,d29c0eb9c94d4225736a15324000b9424518abad,windows_ie,183.131.142.*,94,97,1.0,...,0,13606a7c541dcd9ca1948875a760bb31,227,31,d29e59bf0f7f8243858b8183f14d4412,3358,"10006,10110",windows,ie,336_280
7,0,6,21,17ada861c6ed0616f88312ba0d3d76c0f95b4940,1c2afa6805a0987cd98be4212fcde0f43ea2bdaa,windows_ie,116.252.192.*,238,239,1.0,...,0,00fccc64a1ee2809348509b7ac2a97a5,227,20,b2e35064f3549d447edbbdfb1f707c8c,3427,NaN,windows,ie,300_250
8,0,0,11,6abd8ef3eb678b1afe67dc0cff0aa58413fbf29a,391c06ba8398e249d5c6e370a0ea9d1245ae9b29,windows_chrome,114.226.234.*,80,84,2.0,...,5,f65c8bdb41e9015970bac52baa813239,249,58,9be83422522025c09a44593ecd1af8ca,3476,"10057,10063,10006,10059,10110",windows,chrome,336_280
9,0,5,15,442ffca310e325abc255e8ff97d9812051336acd,e7b2e7f694f4fe57a467fc3879ce4562c797edb4,windows_ie,222.172.73.*,65,73,3.0,...,50,a10c31a8ff5f42930b4c34035e523886,241,55,d29e59bf0f7f8243858b8183f14d4412,3358,"10048,10059,10006,13403,10063,10031,10111",windows,ie,1000_90


In [60]:
# convert numeric (non categorical) fields from int to float (to avoid warnings later)
# TODO

In [61]:
# split into features and labels
X_val = dfVal[['weekday', 'slot_width_height','hour', 'region', 'city', 'slotvisibility', 'slotformat', 'advertiser', 'OS', 'browser',
               'creative',
                  'slotprice', 
                 'usertag']]
# print(X_train.head(5))
y_val = dfVal['click']

In [62]:
X_val_pp = allPreProc.transform(X_val) # note, transform only (no fit)
# adexchange has NaNs in the test dataset (non in train)
#  I have removed this  for now but need to address properly later with additional pre-processing step



In [63]:
allColumns_val = list()
# first columns are the scaled numeric ones, so just numColumns
# for idx in range(len(numColumns)):
#     allColumns.append(numColumns[idx])
for idx in range(len(catgColumns)):
    for subIdx in range(len(allPreProc.named_transformers_['catg'].categories_[idx])):
        allColumns_val.append('_'.join([str(catgColumns[idx]),str(allPreProc.named_transformers_['catg'].categories_[idx][subIdx])]))
for idx in range(len(allPreProc.named_transformers_['tag'].tags)):
    allColumns_val.append('_'.join(['usertag',allPreProc.named_transformers_['tag'].tags[idx]]))
allColumns_val.append('slotprice')    

In [65]:
X_val_pp=pd.DataFrame(X_val_pp)
X_val_pp['slotprice']=dfVal['slotprice']

In [66]:
X_val_pp.shape

(303925, 704)

In [67]:
X_val_pp

,0,1,2,3,4,5,6,7,8,9,...,694,695,696,697,698,699,700,701,702,slotprice
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5
9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,50


In [113]:
# TODO save column layout

In [23]:
# del dfVal
# del X_val
# del X_val_pp
# del y_val
# gc.collect()

56

# Load and run on test data

In [68]:
dfTest = pd.read_csv('/Users/mac/Desktop/multi-agent AI group/test.csv')

In [69]:
dfTest

,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,...,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,keypage,advertiser,usertag
0,0,12,366c563de7d90feb9d4dab53e795a93fb3157387,75045dd2f2136c93fe55fe6c446ec1527ed8f0bb,windows_chrome,27.197.36.*,146,159,3.0,DDTSQuf0MTTNaqKIvMpENpn,...,Astro_F_Rectangle,300,250,0,0,10,00fccc64a1ee2809348509b7ac2a97a5,b2e35064f3549d447edbbdfb1f707c8c,3427,"10024,10077,10075,10063,10031,10102,10006,1304..."
1,3,14,29167d4caa719788b5a342dbaa25151d53121f80,11279eb1f8f7a88f877db911673522b6ff202aa7,android_safari,124.126.227.*,1,1,NaN,d63d96468f66986d5a8eb5225ad0b6f0,...,2745306850,320,50,FirstView,Na,5,11908,NaN,2997,NaN
2,5,19,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,4771a819a3f5b86776d8a9456f4f2506578f78d8,windows_ie,116.116.104.*,27,34,1.0,trqRTJkrBoq7JsNr5SqfNX,...,mm_34022157_3445226_11175100,336,280,2,1,0,77819d3e0b3467fe5c7b16d68ad923a1,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10006,10024,10059,10048,10063,10067,1008..."
3,0,21,844c2da00d45315f20b748ec131c26ee99a7cbc7,5360671379f3204afa9cc0edd8fd2e2096cb09eb,windows_ie,115.46.145.*,238,245,4.0,6d27caf5a202ec085f80af7163ef7cb0,...,210842,960,90,FirstView,Na,30,10717,NaN,2821,"10075,13042,10006,10110,13776,10031,10063"
4,2,20,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,d791b5f6306b9d299f81daa5448ced7e5bfa010f,windows_ie,221.199.203.*,27,35,3.0,3FKElpuEMusyJqKbuKz,...,Fashion_F_Rectangle,300,250,0,0,50,44966cc8da1ed40c95d59e863c8c75f0,43f4a37f42a7c5e6219e2601b26c6976,3386,10063
5,0,19,7493c42f6d8f29d9f414b75c125ce3db40461ef9,aed0d83d9e06589a23e5f02a3e93389f815effc5,windows_ie,122.141.136.*,55,60,3.0,31xSTvprdN1RFt,...,Ent_F_Upright,300,250,0,0,50,a499988a822facd86dd0e8e4ffef8532,bebefa5efe83beee17a3d245e7c5085b,1458,"13496,10093,10057,13866,10063,10110"
6,4,15,819fb6958ea48b10430710d009c57d55a31debcb,1ff9310f6ba827e47af5a7021953a4b9116a6a40,windows_chrome,175.146.224.*,40,48,3.0,ersbQv1RdoTy1m58uG,...,ALLINONE_F_Width1,1000,90,0,0,70,c46090c887c257b61ab1fa11baee91d8,0f951a030abdaedd733ee8d114ce2944,3427,NaN
7,0,0,8660774f6c94e32dfbd46add8a9bc94df3c74130,9a69166c6c091acd1e3fca35f661104a0f756883,windows_ie,202.104.177.*,216,233,1.0,trqRTu1ojqN-gMKzwFnWvpdh,...,mm_10028331_120124_10852228,728,90,1,1,0,48f2e9ba15708c0146bda5e1dd653caa,bebefa5efe83beee17a3d245e7c5085b,1458,NaN
8,4,8,ccc4c75ae4fd33526e5ecdae482c45c1dabc94db,30f29e6da56d60e1a70922cf310df9a01a8c35e3,windows_ie,183.92.64.*,183,186,3.0,ersbQv1RdoTy1m58uG,...,ALLINONE_F_Width1,1000,90,0,0,70,a10c31a8ff5f42930b4c34035e523886,d29e59bf0f7f8243858b8183f14d4412,3358,"10063,10031,10083,10110"
9,1,10,d9a212303746d85553966e902a315ba8812dd138,413efa29cacb96550cc04c2df62e65f72f872812,windows_ie,36.40.65.*,333,334,2.0,trqRTud0B9c7gspy5SqW,...,2069535511,468,60,2,0,5,2f88fc9cf0141b5bbaf251cab07f4ce7,361e128affece850342293213691a043,3386,"10048,10052,13042,10059,10093,11944,10031,1340..."


In [70]:
# Split 'useragent' field into 'OS' and 'browser'
dfn = dfTest['useragent'].str.split('_', expand = True)
dfTest['OS'] = dfn[0]
dfTest['browser'] = dfn[1]
# Create slotsize field
# dfTest['slotsize'] = dfTest['slotheight'] * dfTest['slotwidth']
dfTest['slot_width_height'] = dfTest['slotwidth'].apply(lambda x: str(x)) + "_" + dfTest['slotheight'].apply(lambda x: str(x))
dfTest.drop(['slotwidth', 'slotheight'], axis=1)

,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,...,slotvisibility,slotformat,slotprice,creative,keypage,advertiser,usertag,OS,browser,slot_width_height
0,0,12,366c563de7d90feb9d4dab53e795a93fb3157387,75045dd2f2136c93fe55fe6c446ec1527ed8f0bb,windows_chrome,27.197.36.*,146,159,3.0,DDTSQuf0MTTNaqKIvMpENpn,...,0,0,10,00fccc64a1ee2809348509b7ac2a97a5,b2e35064f3549d447edbbdfb1f707c8c,3427,"10024,10077,10075,10063,10031,10102,10006,1304...",windows,chrome,300_250
1,3,14,29167d4caa719788b5a342dbaa25151d53121f80,11279eb1f8f7a88f877db911673522b6ff202aa7,android_safari,124.126.227.*,1,1,NaN,d63d96468f66986d5a8eb5225ad0b6f0,...,FirstView,Na,5,11908,NaN,2997,NaN,android,safari,320_50
2,5,19,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,4771a819a3f5b86776d8a9456f4f2506578f78d8,windows_ie,116.116.104.*,27,34,1.0,trqRTJkrBoq7JsNr5SqfNX,...,2,1,0,77819d3e0b3467fe5c7b16d68ad923a1,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10006,10024,10059,10048,10063,10067,1008...",windows,ie,336_280
3,0,21,844c2da00d45315f20b748ec131c26ee99a7cbc7,5360671379f3204afa9cc0edd8fd2e2096cb09eb,windows_ie,115.46.145.*,238,245,4.0,6d27caf5a202ec085f80af7163ef7cb0,...,FirstView,Na,30,10717,NaN,2821,"10075,13042,10006,10110,13776,10031,10063",windows,ie,960_90
4,2,20,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,d791b5f6306b9d299f81daa5448ced7e5bfa010f,windows_ie,221.199.203.*,27,35,3.0,3FKElpuEMusyJqKbuKz,...,0,0,50,44966cc8da1ed40c95d59e863c8c75f0,43f4a37f42a7c5e6219e2601b26c6976,3386,10063,windows,ie,300_250
5,0,19,7493c42f6d8f29d9f414b75c125ce3db40461ef9,aed0d83d9e06589a23e5f02a3e93389f815effc5,windows_ie,122.141.136.*,55,60,3.0,31xSTvprdN1RFt,...,0,0,50,a499988a822facd86dd0e8e4ffef8532,bebefa5efe83beee17a3d245e7c5085b,1458,"13496,10093,10057,13866,10063,10110",windows,ie,300_250
6,4,15,819fb6958ea48b10430710d009c57d55a31debcb,1ff9310f6ba827e47af5a7021953a4b9116a6a40,windows_chrome,175.146.224.*,40,48,3.0,ersbQv1RdoTy1m58uG,...,0,0,70,c46090c887c257b61ab1fa11baee91d8,0f951a030abdaedd733ee8d114ce2944,3427,NaN,windows,chrome,1000_90
7,0,0,8660774f6c94e32dfbd46add8a9bc94df3c74130,9a69166c6c091acd1e3fca35f661104a0f756883,windows_ie,202.104.177.*,216,233,1.0,trqRTu1ojqN-gMKzwFnWvpdh,...,1,1,0,48f2e9ba15708c0146bda5e1dd653caa,bebefa5efe83beee17a3d245e7c5085b,1458,NaN,windows,ie,728_90
8,4,8,ccc4c75ae4fd33526e5ecdae482c45c1dabc94db,30f29e6da56d60e1a70922cf310df9a01a8c35e3,windows_ie,183.92.64.*,183,186,3.0,ersbQv1RdoTy1m58uG,...,0,0,70,a10c31a8ff5f42930b4c34035e523886,d29e59bf0f7f8243858b8183f14d4412,3358,"10063,10031,10083,10110",windows,ie,1000_90
9,1,10,d9a212303746d85553966e902a315ba8812dd138,413efa29cacb96550cc04c2df62e65f72f872812,windows_ie,36.40.65.*,333,334,2.0,trqRTud0B9c7gspy5SqW,...,2,0,5,2f88fc9cf0141b5bbaf251cab07f4ce7,361e128affece850342293213691a043,3386,"10048,10052,13042,10059,10093,11944,10031,1340...",windows,ie,468_60


In [146]:
# convert numeric (non categorical) fields from int to float (to avoid warnings later)
# TODO

In [71]:
# split into features and labels
X_test = dfTest[['weekday', 'slot_width_height','hour', 'region', 'city', 'slotvisibility', 'slotformat', 'advertiser', 'OS', 'browser',
               'creative',
                  'slotprice', 
                 'usertag']]
# print(X_train.head(5))

In [72]:
X_test_pp = allPreProc.transform(X_test) # note, transform only (no fit)
# adexchange has NaNs in the test dataset (non in train)
#  I have removed this  for now but need to address properly later with additional pre-processing step



In [73]:
X_test_pp=pd.DataFrame(X_test_pp)
X_test_pp['slotprice']=dfTest['slotprice']

In [75]:
# check correct size
X_test_pp.shape

(303375, 704)

In [84]:
allColumns_test = list()
# first columns are the scaled numeric ones, so just numColumns
# for idx in range(len(numColumns)):
#     allColumns.append(numColumns[idx])
for idx in range(len(catgColumns)):
    for subIdx in range(len(allPreProc.named_transformers_['catg'].categories_[idx])):
        allColumns_test.append('_'.join([str(catgColumns[idx]),str(allPreProc.named_transformers_['catg'].categories_[idx][subIdx])]))
for idx in range(len(allPreProc.named_transformers_['tag'].tags)):
    allColumns_test.append('_'.join(['usertag',allPreProc.named_transformers_['tag'].tags[idx]]))
allColumns_test.append('slotprice')    

In [77]:
X_test_pp

,0,1,2,3,4,5,6,7,8,9,...,694,695,696,697,698,699,700,701,702,slotprice
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70
9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,5


In [78]:
X_test_pp.to_csv('X_test_pp.csv')

In [85]:
len(allColumns_test)

704

In [89]:
from pandas.core.frame import DataFrame
name=DataFrame(allColumns_test)


In [91]:
name.to_csv('column_name.csv')

In [113]:
# TODO save column layout

In [159]:
# del dfTest
# del X_test
# del X_test_pp
# gc.collect()

112